In [56]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

pd.options.display.max_columns = None

In [57]:
gl_19_20 = pd.read_csv('Clean Datasets/Clean-19-20-Game-Log.csv')
gl_19_20.insert(2, 'Season', '2019-2020')
gl_20_21 = pd.read_csv('Clean Datasets/Clean-20-21-Game-Log.csv')
gl_20_21.insert(2, 'Season', '2020-2021')
gl_21_22 = pd.read_csv('Clean Datasets/Clean-21-22-Game-Log.csv')
gl_21_22.insert(2, 'Season', '2021-2022')
gl_22_23 = pd.read_csv('Clean Datasets/Clean-22-23-Game-log.csv')
gl_22_23.insert(2, 'Season', '2022-2023')
gl_23_24 = pd.read_csv('Clean Datasets/Clean-23-24-Game-Log.csv')
gl_23_24.insert(2, 'Season', '2023-2024')

team_opp_19_20 = pd.read_csv('Clean Datasets/Clean-19-20-PerGameTeamAndOpponentStats.csv')
team_opp_20_21 = pd.read_csv('Clean Datasets/Clean-20-21-Team&Opponent-Per-Game.csv')
team_opp_21_22 = pd.read_csv('Clean Datasets/Clean-21-22-Team&Opp-Per-Game.csv')
team_opp_22_23 = pd.read_csv('Clean Datasets/Clean-22-23-Team&Opponents.csv')
team_opp_23_24 = pd.read_csv('Clean Datasets/Clean-23-24-Team&Opp-Per-Game.csv')

roster_19_20 = pd.read_csv('Clean Datasets/Clean-19-20-roster.csv')
roster_20_21 = pd.read_csv('Clean Datasets/Clean-20-21-Roster.csv')
roster_21_22 = pd.read_csv('Clean Datasets/Clean-21-22-Roster.csv')
roster_22_23 = pd.read_csv('Clean Datasets/Clean-22-23-Roster.csv')
roster_23_24 = pd.read_csv('Clean Datasets/Clean-23-24-Roster.csv')

In [58]:
# Correct the Win Streak and Lose Streak feature

def add_streak(game_log):
    # Ensure the Date column is in datetime format and sorted
    game_log['Date'] = pd.to_datetime(game_log['Date'])
    game_log = game_log.sort_values(by='Date').reset_index(drop=True)

    # Initialize streak columns
    win_streak = []
    lose_streak = []

    # Track streak counters
    current_win_streak = 0
    current_lose_streak = 0

    # Iterate over each row in the game log
    for _, row in game_log.iterrows():
        if row['Result'] == 'W':  # If the team won
            win_streak.append(current_win_streak)
            lose_streak.append(current_lose_streak)
            current_win_streak += 1
            current_lose_streak = 0
        else:  # If the team lost
            win_streak.append(current_win_streak)
            lose_streak.append(current_lose_streak)
            current_lose_streak += 1
            current_win_streak = 0

    # Add streak columns to the DataFrame
    game_log['Win Streak'] = win_streak
    game_log['Lose Streak'] = lose_streak

    return game_log

gl_19_20 = add_streak(gl_19_20)
gl_20_21 = add_streak(gl_20_21)
gl_21_20 = add_streak(gl_20_21)
gl_21_22 = add_streak(gl_21_22)
gl_22_23 = add_streak(gl_22_23)
gl_23_24 = add_streak(gl_23_24)

In [59]:
# Adding a feature that tells us how much rest the Fighting Illini had before their current game

def add_rest_time(game_log):
    # Ensure the Date column is in datetime format and sorted
    game_log['Date'] = pd.to_datetime(game_log['Date'])
    game_log = game_log.sort_values(by='Date').reset_index(drop=True)
    
    # Calculate rest days
    game_log['Rest Days'] = game_log['Date'].diff().dt.days.fillna(0).astype(int)
    
    # Fix any negative or unreasonable rest days
    game_log['Rest Days'] = game_log['Rest Days'].apply(lambda x: max(x, 0))  # No negative rest days

    # Insert Rest Days at the 7th index position
    game_log.insert(8, 'Rest Days', game_log.pop('Rest Days'))
    
    return game_log

gl_19_20 = add_rest_time(gl_19_20)
gl_20_21 = add_rest_time(gl_20_21)
gl_21_20 = add_rest_time(gl_20_21)
gl_21_22 = add_rest_time(gl_21_22)
gl_22_23 = add_rest_time(gl_22_23)
gl_23_24 = add_rest_time(gl_23_24)

In [60]:
# Adding a feature that calculates the rolling average of our stats

# List of statistical columns for which to calculate rolling averages
stats_columns = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
                 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 
                 'OPP FG', 'OPP FGA', 'OPP FG%', 'OPP 3P', 'OPP 3PA', 'OPP 3P%',
                 'OPP FT', 'OPP FTA', 'OPP FT%', 'OPP ORB', 'OPP TRB', 'OPP AST',
                 'OPP STL', 'OPP BLK', 'OPP TOV', 'OPP PF']

# Define a function to add rolling averages
def add_rolling_averages(df, window=5):
    for col in stats_columns:
        rolling_col_name = f"{col}_rolling_{window}"
        df[rolling_col_name] = df[col].rolling(window=window, min_periods=1).mean()
    return df

# Apply to each dataset
gl_19_20 = add_rolling_averages(gl_19_20)
gl_20_21 = add_rolling_averages(gl_20_21)
gl_21_22 = add_rolling_averages(gl_21_22)
gl_22_23 = add_rolling_averages(gl_22_23)
gl_23_24 = add_rolling_averages(gl_23_24)

In [ ]:
all_gl = pd.concat([gl_19_20, gl_20_21, gl_21_22, gl_22_23, gl_23_24], ignore_index=True)
all_gl = all_gl.drop(columns='Unnamed: 0')
all_gl

In [62]:
                # all_gl.to_csv(r'/Users/isaac/Desktop/personal/datadive24/Clean Datasets/All-Clean-Game-Logs.csv')

In [ ]:
# Observe if Rest Days Were Correctly Calculated
all_gl['Rest Days'].unique()

In [ ]:
# Rename duplicate column names to be more concise
all_gl.rename(columns={'PF': 'Points For', 'PA' : 'Points Against', 'PF.1' : 'PF'}, inplace=True)
all_gl

In [ ]:
all_gl.isna().sum()

In [ ]:
# Find the row with missing values
lindenwood = all_gl.loc[all_gl['Conf'].isna()]
lindenwood

In [ ]:
# Fill in missing conference value
lindenwood['Conf'] = 'OVC'
lindenwood

In [ ]:
# Fill in missing SRS value with an estimate
lindenwood['SRS'] = 42
lindenwood

In [69]:
all_gl.loc[all_gl['Conf'].isna()] = lindenwood

In [ ]:
all_gl.isna().sum()

In [ ]:
# Calculate the result counts
result_counts = all_gl['Result'].value_counts().reset_index()
result_counts.columns = ['Result', 'Frequency']

sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 6))

sns.barplot(data=result_counts, x='Result', y='Frequency', palette=['#E84A27', '#13294b'])

plt.xlabel('Game Result', fontsize=14, labelpad=10)
plt.ylabel('Frequency', fontsize=14, labelpad=10)
plt.title('Frequency of Wins and Losses Over Past 5 Seasons', fontsize=16, pad=15)

# Adding value annotations on top of the bars
for index, row in result_counts.iterrows():
    plt.text(row.name, row.Frequency + 2, row.Frequency, ha='center', va='bottom', fontsize=12)

plt.show()


In [ ]:
# Observe how well we do against ranked teams
plt.figure(figsize=(10, 6))

# Create a grouped bar plot using 'Result' and 'Ranked Match' columns
plot = sns.countplot(data=all_gl, x='Result', hue='Ranked Match', palette=['#E84A27', '#13294b'])

plt.xlabel('Game Result')
plt.ylabel('Frequency')
plt.title('Wins and Losses by Ranked vs. Non-Ranked Opponents')
plt.legend(title='Ranked Match', labels=['Non-Ranked', 'Ranked'])

# Adds count at the top of the bar
for bar in plot.containers:
    plot.bar_label(bar, label_type='edge', padding=5, fontsize=10)

plt.show()

In [ ]:
stats = [
    'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 
    'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 
    'OPP FG', 'OPP FGA', 'OPP FG%', 'OPP 3P', 'OPP 3PA', 'OPP 3P%', 
    'OPP FT', 'OPP FTA', 'OPP FT%', 'OPP ORB', 'OPP TRB', 
    'OPP AST', 'OPP STL', 'OPP BLK', 'OPP TOV', 'OPP PF'
]

# Calculate the mean of each stat grouped by 'Ranked Match'
avg_stats = all_gl.groupby('Ranked Match')[stats].mean().reset_index()

# Reshape the data to have each stat as a separate entry for grouped plotting
avg_stats_melted = avg_stats.melt(id_vars='Ranked Match', var_name='Stat', value_name='Average')

plt.figure(figsize=(27, 13))
plot = sns.barplot(data=avg_stats_melted, x='Stat', y='Average', hue='Ranked Match', palette=['#E84A27', '#13294b'])

plt.xlabel('Game Stats')
plt.ylabel('Average Value')
plt.title('Average Game Stats by Ranked vs. Non-Ranked Opponents')
plt.legend(title='Ranked Match', labels=['Non-Ranked', 'Ranked'])

# Adds average value at the top of each bar
for bar in plot.containers:
    plot.bar_label(bar, fmt="%.1f", label_type='edge', padding=5, fontsize=10)

plt.xticks(rotation=45)
plt.show()

In [ ]:
# What about for Conference vs. Non-Conference games
conf_gl = all_gl.copy()

conf_gl['Big_Ten_Opponent'] = conf_gl['Conf'].apply(lambda x: 'Big Ten' if x == 'Big Ten' else 'Non-Big Ten')

plt.figure(figsize=(10, 6))

plot = sns.countplot(data=conf_gl, x='Result', hue='Big_Ten_Opponent', palette=['#E84A27', '#13294b'])

plt.xlabel('Game Result')
plt.ylabel('Frequency')
plt.title('Wins and Losses by Big Ten vs. Non-Big Ten Opponents')
plt.legend(title='Conference', labels=['Non-Big Ten', 'Big Ten'])

# Add counts at the top of each bar
for bar in plot.containers:
    plot.bar_label(bar, label_type='edge', padding=5, fontsize=10)

plt.show()

In [ ]:
# What about for Home vs. Away games
plt.figure(figsize=(10, 6))

plot = sns.countplot(data=all_gl, x='Result', hue='Home Game', palette=['#E84A27', '#13294b'])

plt.xlabel('Game Result', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Wins and Losses by Home vs. Away Games', fontsize=14)
plt.legend(title='Location', labels=['Home', 'Away'], fontsize=10)

# Add counts at the top of each bar
for bar in plot.containers:
    plot.bar_label(bar, label_type='edge', padding=5, fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Observe how the relationship between rest days and game results
plt.figure(figsize=(12, 6))

# Group data by Rest Days and Result, then count occurrences
rest_days_result = all_gl.groupby(['Rest Days', 'Result']).size().reset_index(name='Frequency')

plot = sns.barplot(data=rest_days_result, x='Rest Days', y='Frequency', hue='Result', palette=['#E84A27', '#13294b'])

plt.xlabel('Rest Days', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Frequency of Game Results by Rest Days', fontsize=14)
plt.legend(title='Game Result', labels=['Loss', 'Win'], fontsize=10)

# Add value labels to each bar
for bar in plot.containers:
    plot.bar_label(bar, label_type='edge', padding=3, fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Create bins for Rest Days and apply to our graph to better observe any trends
bins = [0, 1, 3, 5, 8, 100] 
labels = ['0', '1-2', '3-4', '5-7', '8+'] 
all_gl['Rest_Days_Binned'] = pd.cut(all_gl['Rest Days'], bins=bins, labels=labels, right=False)

plt.figure(figsize=(12, 7))

plot = sns.countplot(data=all_gl, x='Rest_Days_Binned', hue='Result', palette=['#E84A27', '#13294b'])

plt.xlabel('Rest Days (Binned)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Game Results by Rest Days', fontsize=14)
plt.legend(title='Game Result', labels=['Loss', 'Win'])

# Add counts at the top of each bar
for bar in plot.containers:
    plot.bar_label(bar, label_type='edge', padding=5, fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# show the teams we have highest chance of winning against from lowest to highest

big_ten_schools = conf_gl[conf_gl['Big_Ten_Opponent'] == 'Big Ten']

big_ten_matchups = big_ten_schools.groupby('Opponent').agg(Total_Games=('Opponent', 'size'), 
    Wins=('Result', lambda x: (x == 'W').sum())).reset_index()

big_ten_matchups['Win Percentage'] = (big_ten_matchups['Wins'] / big_ten_matchups['Total_Games']) * 100

big_ten_matchups_sorted = big_ten_matchups.sort_values(by='Win Percentage', ascending=False)

big_ten_matchups_sorted

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=big_ten_matchups_sorted, x='Opponent', y='Win Percentage', palette=['#E84A27', '#13294b'])

plt.title('Win Percentage against Big Ten Schools')
plt.xlabel('Opponent')
plt.ylabel('Win Percentage (%)')
plt.xticks(rotation=90)

plt.show()

In [ ]:
# Make a numeric only game log data frame for our correlation matrix
num_gl = all_gl.copy()
num_gl['Result'] = num_gl['Result'].apply(lambda x: 1 if x == 'W' else 0)
num_gl['Ranked Match'] = num_gl['Ranked Match'].apply(lambda x: 1 if x == True else 0)
num_gl['Home Game'] = num_gl['Home Game'].apply(lambda x: 1 if x == True else 0)
num_gl['Type'] = num_gl['Type'].astype('category').cat.codes
num_gl['Conf'] = num_gl['Conf'].astype('category').cat.codes
num_gl = num_gl.drop(columns=['Season','Date','Time','Rank','Opponent', 'Rest_Days_Binned'])
num_gl

In [ ]:
# Create correlation matrix
gl_corr = num_gl.corr()
gl_corr

In [ ]:
# The variables that have the most affect on the Result column in order
pd.set_option('display.max_rows', None)
result_corr = gl_corr['Result'].drop('Result')
result_corr = result_corr.abs().sort_values(ascending=False)
result_corr = result_corr.reset_index()
result_corr.columns = ['Variable', 'Correlation with Result']
result_corr

In [ ]:
# Visualization to easily view what variables had the highest correlation with result
plt.figure(figsize=(12, 8))
sns.barplot(data=result_corr, x='Variable', y='Correlation with Result', color='orange')
plt.xticks(rotation=90)
plt.title('Correlation of Variables with Game Result')
plt.xlabel('Variable')
plt.ylabel('Correlation with Result')
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore') #mutes warnings for 2021 and 2122 

s1920 = ['19-20', '19-20', '19-20', '19-20', '19-20', '19-20', '19-20', 
         '19-20', '19-20', '19-20' , '19-20', '19-20', '19-20', '19-20']
roster_19_20['Season'] = s1920 # add season columns to rosters for analysis later

s2021 = ['20-21', '20-21', '20-21', '20-21', '20-21', '20-21', '20-21'
         '20-21', '20-21', '20-21', '20-21', '20-21', '20-21', '20-21', '20-21', '20-21']
roster_20_21['Season'] = s2021
roster_20_21['Season'][6] = '20-21'

s2122 = ['21-22', '21-22', '21-22', '21-22', '21-22', '21-22', '21-22'
         '21-22', '21-22', '21-22', '21-22', '21-22', '21-22', '21-22', '21-22', '21-22']
roster_21_22['Season'] = s2122
roster_21_22['Season'][6] = '21-22'


s2223 = ['22-23', '22-23', '22-23', '22-23', '22-23',
         '22-23', '22-23', '22-23', '22-23', '22-23', 
         '22-23', '22-23', '22-23', '22-23', '22-23']
roster_22_23['Season'] = s2223

s2324 = ['23-24', '23-24', '23-24', '23-24', '23-24', '23-24',
         '23-24', '23-24', '23-24', '23-24', '23-24', '23-24'
         ,'23-24']
roster_23_24['Season'] = s2324


all_rosters = pd.concat([roster_19_20, roster_20_21, roster_21_22, roster_22_23, roster_23_24], ignore_index= True)
count_players = all_rosters['Player'].value_counts()
multiple_season = count_players[count_players > 1].index.to_list()
attributes = ['Player', 'Season', 'PTS', 'AST']
roster_performance = all_rosters[all_rosters['Player'].isin(multiple_season)][attributes]
player_pts = roster_performance.pivot(index = 'Season', columns = 'Player', values = 'PTS')
plt.figure(figsize = (10,8))
player_pts
for player in player_pts.columns :
    plt.plot(player_pts.index, player_pts[player], marker = 'o', label = player)

plt.title('Points Per Game Trends for Players Across Seasons')
plt.xlabel('Season')
plt.ylabel('Points Per Game')
plt.xticks(rotation=45)
plt.legend(title='Player', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
if 'Pos' in all_rosters.columns and 'PTS' in all_rosters.columns: 
    position = all_rosters['Pos'].unique()
    for pos in position:
        position_data = all_rosters[all_rosters['Pos'] == pos]
        pts_for_pos = position_data.pivot(index = 'Season', columns = 'Player', values = 'PTS')
        plt.figure(figsize = (10,6))
        for player in pts_for_pos.columns: 
            plt.plot(pts_for_pos.index, pts_for_pos[player], marker = 'o', label = player)
        plt.title(f'Points Per Game Trends by Season for {pos} Position')
        plt.xlabel('Season')
        plt.ylabel('Points Per Game')
        plt.legend(title='Player', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [86]:
#Code below makes a linear regression model predicting points based on various metrics 

all_rosters[['Feet', 'Inches']] = all_rosters['Height'].str.split('-', expand= True)
all_rosters['Feet'] = all_rosters['Feet'].astype(int)
all_rosters['Inches'] = all_rosters['Inches'].astype(int)
all_rosters['Height(inches)'] = all_rosters['Feet'] * 12 + all_rosters['Inches']
all_rosters.drop(columns = ['Unnamed: 0']) #converts the height to inches for linear regression model and also cleans it up a little
all_rosters['Predicted PTS'] = all_rosters['PTS']

In [ ]:
x = all_rosters[['PTS', 'AST', 'Height(inches)', 'Weight']]
y = all_rosters['Predicted PTS']

drop_na = pd.concat([x, y], axis = 1).dropna()

x_c = drop_na[['PTS', 'AST', 'Height(inches)', 'Weight']]
y_c = drop_na['Predicted PTS']

x_train, x_test, y_train, y_test = train_test_split(x_c, y_c, test_size = 0.3, random_state = 42)

model = LinearRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
predictions

In [ ]:
y_test

In [ ]:
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print("MSE = " + mse.astype(str) + "\n" + "R^2 = " + r2.astype(str))

In [ ]:
all_rosters

In [ ]:
selected_columns = [
    'Player', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', 
    '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'STL', 
    'BLK', 'TOV', 'PTS', 'Height(inches)'
]

df_selected_outliers = all_rosters[selected_columns]
df_selected_outliers

In [ ]:
# looking for outliers within specific features

plt.figure(figsize=(15, 10))
df_selected_outliers.boxplot()
plt.xticks(rotation = 90) 
plt.title("Boxplot of Selected Features to Identify Outliers")
plt.show()

In [ ]:
from scipy.stats import zscore

z_scores = df_selected_outliers.select_dtypes(include=['float64', 'int']).apply(zscore)

outliers = (z_scores.abs() > 3).any(axis=1)
df_outliers = df_selected_outliers[outliers]
df_outliers

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_selected_outliers[['MP', 'PTS']])
plt.title("Boxplot of Minutes Played and Points")
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.show()

In [ ]:
numeric_roster = all_rosters.select_dtypes(include=[np.number])
corr = numeric_roster.corr()
plt.figure(figsize=(12, 10))

sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm',
            xticklabels=corr.columns, yticklabels=corr.columns)

plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title('Correlation Matrix Heatmap')
plt.show()